In [1]:
#1 - libraries
import drms
import os
import pandas as pd
import dask.dataframe as dd
import datetime as dt
import numpy as np
import csv

from sunpy.time import TimeRange
import sunkit_instruments.goes_xrs

import tensorflow as tf

from datetime import datetime, date, time, timedelta

C:\Users\Juliana\AppData\Roaming\Python\Python39\site-packages\requests\__init__.py:109: RequestsDependencyWarning: urllib3 (2.0.2) or chardet (5.1.0)/charset_normalizer (3.1.0) doesn't match a supported version!
  warnings.warn(


In [2]:
#2 - data range collect
start_date = '2010-01-01'
end_date = '2024-04-30'

In [ ]:
#3 - data colect from GOES

if os.path.isfile('data/goes_data_'+str(start_date)+'_'+str(end_date)+'.npy'):
    goes_events = np.load('data/goes_data_'+str(start_date)+'_'+str(end_date)+'.npy', allow_pickle=True, encoding='latin1')      
else:
    print("Download new GOES data")
    time_range = TimeRange(start_date, end_date)
    goes_events = sunkit_instruments.goes_xrs.get_goes_event_list(time_range,"M1") 

    np.save('data/goes_data_'+str(start_date)+'_'+str(end_date)+'.npy', goes_events)

    lista_goes = []

    for ge in goes_events:
        item_goes = []
        item_goes.append(ge['event_date'])
        item_goes.append(ge['start_time'])
        item_goes.append(ge['peak_time'])
        item_goes.append(ge['end_time'])
        item_goes.append(ge['goes_class'])
        item_goes.append(ge['goes_location'])
        item_goes.append(ge['noaa_active_region'])
        lista_goes.append(item_goes)


    df = pd.DataFrame(lista_goes)
    df.to_csv('data/goes_data_'+str(start_date)+'_'+str(end_date)+'.csv')


    
# map active regions  NOAA e HARPNUMs 
if os.path.isfile('data/all_harps_with_noaa_ars.txt'):
    num_mapper = pd.read_csv('data/all_harps_with_noaa_ars.txt', sep=' ')
else:
    num_mapper = pd.read_csv('http://jsoc.stanford.edu/doc/data/hmi/harpnum_to_noaa/all_harps_with_noaa_ars.txt',sep=' ')

In [4]:
#4 - general functions

def from_tai_time(tstr):
    """
    Convert the time string to a dt object
    """
    return dt.datetime.strptime(tstr, "%Y.%m.%d_%H:%M:%S_TAI")

def to_tai_time(date):
    """
    Convert from a dt object to a time string object
    """
    #original  .strftime
    return date.strftime("%Y.%m.%d_%H:%M:%S_TAI")

def round_up(time):
    """
    Rounds up the time to the next time divisible by 12 minutes (JSOC data is saved every 12 minutes)
    
    Args:
    time := td object
    """
    #remainder = time.minute % 12
    remainder =  (int( str(time)[14:16] )) % 12

    if remainder != 0:
        time = time + dt.timedelta(minutes=12-remainder)
    return time


def convert_noaa_to_harpnum(noaa_ar):
    """
    Converts from a NOAA Active Region to a HARPNUM
    Returns harpnum if present, else None if there are no matching harpnums
    
    Args:
    """
    idx = num_mapper[num_mapper['NOAA_ARS'].str.contains(str(int(noaa_ar)))]
    return None if idx.empty else idx.HARPNUM.values[0]


In [ ]:
#5 - collect goes-data from JSOC - events M, X


for event in goes_events:
    harpnum = convert_noaa_to_harpnum(event['noaa_active_region'])
    goes_class = event['goes_class']
   
    
    if harpnum == None:
        print('there are no matching HARPNUMs for', str(int(event['noaa_active_region'])))
        continue

    #Subtracts the forecast window from the GOES event date
    date_event = event['peak_time'].to_datetime() - dt.timedelta(hours=6)
 
    #connect SHARP dataset
    c = drms.Client(server='jsoc', email='julianasabfer@gmail.com', verbose=False, debug=False)
    keys, segments = c.query('hmi.sharp_720s['+str(harpnum)+']['+str(date_event)+'] ', key='OBS_VR,QUALITY,DATE,DATE_S,DATE_B,DATE__OBS,DATE-OBS,T_OBS,T_REC,harpnum,USFLUX,MEANGAM,MEANGBT,MEANGBZ,MEANGBH,MEANJZD,TOTUSJZ,MEANALP,MEANJZH,TOTUSJH,ABSNJZH,SAVNCPP,MEANPOT,TOTPOT,MEANSHR,SHRGT45,R_VALUE,AREA_ACR', seg='Br')
    df = pd.DataFrame(keys)

   
    if keys.empty:
        print("there are no matching data for datetime: ", str(date_event))
        continue

    
    keys["Class_Flare"] = [goes_class]

    #Assigns binary class to events
    if "A" in goes_class or "B" in goes_class or "C" in goes_class:
        keys["Class"] = [0]
    elif "M" in goes_class or "X" in goes_class:
        keys["Class"] = [1]

    #save results
    if(os.path.isfile('data/sharp_mx_'+str(start_date)+'_'+str(end_date)+'.csv')):
        keys.to_csv('data/sharp_mx_'+str(start_date)+'_'+str(end_date)+'.csv', mode='a', index= False,header=False)
    else:
        keys.to_csv('data/sharp_mx_'+str(start_date)+'_'+str(end_date)+'.csv', index= False, header=True)




In [ ]:
#6 - data colect from JSOC - all events - flare and no-flare events

start = dt.datetime.strptime(start_date, '%Y-%m-%d') 
end = dt.datetime.strptime(end_date, '%Y-%m-%d')

size = (end-start).days * 4

all_times = {start + dt.timedelta(hours=i*6) for i in range(size)}
all_times = list(all_times)
all_times.sort()


for al in all_times:
    
    dataformat = str(al)

    c = drms.Client(server='jsoc', email='julianasabfer@gmail.com', verbose=False, debug=False)
    keys, segments = c.query('hmi.sharp_720s[]['+dataformat+'] ', key='OBS_VR,QUALITY,DATE,DATE_S,DATE_B,DATE__OBS,DATE-OBS,T_OBS,T_REC,harpnum,USFLUX,MEANGAM,MEANGBT,MEANGBZ,MEANGBH,MEANJZD,TOTUSJZ,MEANALP,MEANJZH,TOTUSJH,ABSNJZH,SAVNCPP,MEANPOT,TOTPOT,MEANSHR,SHRGT45,R_VALUE,AREA_ACR', seg='Br')

    
    if keys.empty:
        print("there are no matching data for datetime: ", str(date_event))
        continue

    
    keys["Class_Flare"] = "NF"
    keys["Class"] = 0


    if(os.path.isfile('data/sharp_no-class_'+str(start_date)+'_'+str(end_date)+'.csv')):
        keys.to_csv('data/sharp_no-class_'+str(start_date)+'_'+str(end_date)+'.csv', mode='a', index= False,header=False)
    else:
        keys.to_csv('data/sharp_no-class_'+str(start_date)+'_'+str(end_date)+'.csv', index= False, header=True)



In [ ]:
#7  - Set rows corresponding to solar flare events and remove rows without an active region (ARs)

raw_df = pd.read_csv(f'data/sharp_no-class_{start_date}_{end_date}.csv', sep=",")
raw_df['T_REC'] = pd.to_datetime(raw_df['T_REC'], format='%Y.%m.%d_%H:%M:%S_TAI')

# Loop over GOES events
for event in goes_events:

    event_date = round_up(event['peak_time'].to_datetime())
    harpnum = convert_noaa_to_harpnum(event['noaa_active_region'])

    # Short window: -6h to event
    start_time = event_date - dt.timedelta(hours=6)
    end_time = event_date

    # Filter with harp_num
    mask_harp = (raw_df["T_REC"].between(start_time, end_time)) & (raw_df["harpnum"] == harpnum)
    num_matches_harp = mask_harp.sum()

    if num_matches_harp > 0:
        raw_df.loc[mask_harp, "Class"] = 1
        print(f"[FLARE] {num_matches_harp} lines marked as Class=1 for HARP {harpnum} ({start_time} → {end_time})")

    else:
        # Same window but without harp_num
        mask_all = raw_df["T_REC"].between(start_time, end_time)
        num_matches_all = mask_all.sum()

        if num_matches_all > 0:
            raw_df.loc[mask_all, "Class"] = 2
            print(f"[ALT] No records for HARP {harpnum}, but {num_matches_all} lines marked as Class=2 (no harp filter) between {start_time} → {end_time}")
        else:
            print(f"[WARN] No data found even without harp filter ({start_time} → {end_time})")


# After loop: remove Class=2 records
num_to_remove = (raw_df["Class"] == 2).sum()
print(f"[INFO] Removing {num_to_remove} lines with Class=2...")

df_final = raw_df.loc[raw_df["Class"] != 2].copy()

print(f"[INFO] Remaining lines after removal: {len(df_final)}")

df_final.to_csv('data/sharp_no-flare_'+str(start_date)+'_'+str(end_date)+'.csv', index= False, header=True)

In [ ]:
#8 create full dataset

#load file solar flare events sharp
df_events_abcmx = pd.read_csv('data/sharp_mx_'+str(start_date)+'_'+str(end_date)+'.csv', sep=",")
date1 = pd.to_datetime(df_events_abcmx ['T_REC'], errors='coerce', format='%Y-%m-%d %H:%M:%S')
date2 = pd.to_datetime(df_events_abcmx ['T_REC'], errors='coerce', format='%Y-%m-%d')
df_events_abcmx ['T_REC'] = date1.fillna(date2)

#load file no-evets sharp
df_events_noflare = pd.read_csv('data/sharp_no-flare_'+str(start_date)+'_'+str(end_date)+'.csv', sep=",")
date1 = pd.to_datetime(df_events_noflare['T_REC'], errors='coerce', format='%Y-%m-%d %H:%M:%S')
date2 = pd.to_datetime(df_events_noflare['T_REC'], errors='coerce', format='%Y-%m-%d')
df_events_noflare['T_REC'] = date1.fillna(date2)


#concat events

print("Nº ABCMX: ", len(df_events_abcmx))
print("Nº NF: ", len(df_events_noflare))

df_all_events = pd.concat([df_events_abcmx, df_events_noflare])
print("All events: ", len(df_all_events))


#drop duplicates rows
df_all_events = df_all_events.drop_duplicates()
print("All events after removed duplicates:", len(df_all_events))

#Discart bad rows
mask = (df_all_events['QUALITY'] >= 65536) | (df_all_events['OBS_VR'].abs() >= 3500)
df_all_events_discart = df_all_events.loc[~mask]

print("All events after QUALITY and OBS_VR: ", len(df_all_events_discart))


#Discart null rows
df_all_events_discart.replace([np.inf, -np.inf], np.nan, inplace=True)
df_all_events_discart.dropna(subset=df_all_events_discart.columns, inplace=True)
df_all_events_discart.drop

print("All events after dicart null rows: ", len(df_all_events_discart))

df_all_events.to_csv('data/sharp_mx-nf-all_'+str(start_date)+'_'+str(end_date)+'.csv', index= False, header=True)